In [13]:
# -----------------------------
# Import libraries
# -----------------------------
import os
import json
import requests
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict
from tqdm import tqdm
import plotly.express as px
import defeatbeta_api
from defeatbeta_api.data.ticker import Ticker
import numpy as np
# import torch
from itertools import combinations
from sklearn.preprocessing import OneHotEncoder, StandardScaler

start_date = pd.to_datetime("31-12-2017", dayfirst=True)

In [12]:
!pip install plotly
!pip install scikit-learn


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 12.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 10.1 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


# Constituent list

In [ ]:
# -----------------------------
# Retrieve current S&P 500 constituents
# -----------------------------
def get_current_constituents():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#Selected_changes_to_the_list_of_S&P_500_components"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # The 1st table contains the current list
    df = pd.read_html(response.text, header=0)[0]
    df = df[["Symbol", "Security"]]
    return df

# -----------------------------
# Retrieve historical composition changes from Wikipedia
# -----------------------------
def get_historical_changes():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#Selected_changes_to_the_list_of_S&P_500_components"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # The 2nd table contains the historical changes
    changes = pd.read_html(response.text, header=[0, 1])[1]

    # Flatten multi-level columns and normalize
    changes.columns = [" ".join(col).strip() for col in changes.columns.values]
    changes = changes.rename(columns={"Effective Date Effective Date": "Date"})
    changes["Date"] = pd.to_datetime(changes["Date"], errors="coerce")

    return changes.dropna(subset=["Date"]).sort_values("Date")

# -----------------------------
# Build a timeline of index members over time
# -----------------------------
def build_constituents_timeline(current, changes):
    members = set(current["Symbol"])
    timeline = defaultdict(set)

    # Iterate backwards to reconstruct past compositions
    for _, row in changes.sort_values("Date", ascending=False).iterrows():
        date = row["Date"]
        if pd.notna(row.get("Added Ticker")):
            members.discard(row["Added Ticker"])
        if pd.notna(row.get("Removed Ticker")):
            members.add(row["Removed Ticker"])
        timeline[date.strftime("%d-%m-%Y")] = members.copy()

    return dict(sorted(timeline.items()))

# -----------------------------
# Save timeline as JSON
# -----------------------------
def save_timeline(timeline, filename="sp500_timeline.json"):
    serializable = {d: sorted(list(tickers)) for d, tickers in timeline.items()}
    with open(filename, "w") as f:
        json.dump(serializable, f, indent=2)
    print(f"Timeline saved to {filename}")

# -----------------------------
# Main execution
# -----------------------------
current = get_current_constituents()
changes = get_historical_changes()
timeline = build_constituents_timeline(current, changes)
save_timeline(timeline)

# -----------------------------
# Load and analyze timeline
# -----------------------------
with open("sp500_timeline.json", "r") as f:
    data = json.load(f)

timeline = {datetime.strptime(k, "%d-%m-%Y"): set(v) for k, v in data.items()}
if not timeline:
    raise SystemExit("No snapshots found in sp500_timeline.json")

# Filter snapshots from the last 5 years
cutoff_date = datetime.now() - timedelta(days=5*365)
recent_snapshots = [s for d, s in timeline.items() if d >= cutoff_date and s]

if not recent_snapshots:
    raise SystemExit("No snapshots found in the last 5 years")

# Compute tickers present in all recent snapshots
present_last_5_years = set.intersection(*recent_snapshots)

print(f"Tickers consistently present in the last 5 years: {len(present_last_5_years)}")
print(sorted(present_last_5_years))

In [14]:
# -----------------------------
# Ticker list
# -----------------------------

# List of tickers
tickers = ['A', 'AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIZ', 'AJG', 'AKAM',
           'ALB', 'ALGN', 'ALL', 'ALLE', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'AON', 'AOS', 'APA', 'APD', 'APH',
           'APTV', 'ARE', 'ATO', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BALL', 'BAX', 'BBY', 'BDX', 'BEN', 'BF.B', 'BIIB', 'BK',
           'BKNG', 'BKR', 'BLK', 'BMY', 'BR', 'BRK.B', 'BSX', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS',
           'CDW', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COO', 'COP',
           'COR', 'COST', 'CPAY', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTRA', 'CTSH', 'CTVA', 'CVS', 'CVX', 'D', 'DAL', 'DAY', 'DD', 'DE', 'DG',
           'DGX', 'DHI', 'DHR', 'DIS', 'DLR', 'DLTR', 'DOC', 'DOV', 'DOW', 'DPZ', 'DRI', 'DTE', 'DUK', 'DVA', 'DVN', 'DXCM', 'EA', 'EBAY', 'ECL', 'ED', 'EFX',
           'EG', 'EIX', 'EL', 'ELV', 'EMN', 'EMR', 'EOG', 'EQIX', 'EQR', 'ES', 'ESS', 'ETN', 'ETR', 'EVRG', 'EW', 'EXC', 'EXPD', 'EXPE', 'EXR', 'F', 'FANG',
           'FAST', 'FCX', 'FDX', 'FE', 'FFIV', 'FI', 'FIS', 'FITB', 'FOX', 'FOXA', 'FRT', 'FTNT', 'FTV', 'GD', 'GE', 'GEN', 'GILD', 'GIS', 'GL', 'GLW', 'GM',
           'GOOG', 'GOOGL', 'GPC', 'GPN', 'GRMN', 'GS', 'GWW', 'HAL', 'HAS', 'HBAN', 'HCA', 'HD', 'HIG', 'HII', 'HLT', 'HOLX', 'HON', 'HPE', 'HPQ', 'HRL', 'HSIC',
           'HST', 'HSY', 'HUM', 'HWM', 'IBM', 'ICE', 'IDXX', 'IEX', 'IFF', 'INCY', 'INTC', 'INTU', 'IP', 'IPG', 'IQV', 'IR', 'IRM', 'ISRG', 'IT', 'ITW', 'IVZ', 'J',
           'JBHT', 'JCI', 'JKHY', 'JNJ', 'JPM', 'K', 'KEY', 'KEYS', 'KHC', 'KIM', 'KLAC', 'KMB', 'KMI', 'KMX', 'KO', 'KR', 'L', 'LDOS', 'LEN', 'LH', 'LHX', 'LIN',
           'LKQ', 'LLY', 'LMT', 'LNT', 'LOW', 'LRCX', 'LUV', 'LVS', 'LW', 'LYB', 'LYV', 'MA', 'MAA', 'MAR', 'MAS', 'MCD', 'MCHP', 'MCK', 'MCO', 'MDLZ', 'MDT', 'MET',
           'META', 'MGM', 'MHK', 'MKC', 'MKTX', 'MLM', 'MMC', 'MMM', 'MNST', 'MO', 'MOS', 'MPC', 'MRK', 'MS', 'MSCI', 'MSFT', 'MSI', 'MTB', 'MTD', 'MU', 'NCLH', 'NDAQ',
           'NEE', 'NEM', 'NFLX', 'NI', 'NKE', 'NOC', 'NOW', 'NRG', 'NSC', 'NTAP', 'NTRS', 'NUE', 'NVDA', 'NVR', 'NWS', 'NWSA', 'O', 'ODFL', 'OKE', 'OMC', 'ORCL',
           'ORLY', 'OTIS', 'OXY', 'PAYC', 'PAYX', 'PCAR', 'PEG', 'PEP', 'PFE', 'PFG', 'PG', 'PGR', 'PH', 'PHM', 'PKG', 'PLD', 'PM', 'PNC', 'PNR', 'PNW', 'POOL',
           'PPG', 'PPL', 'PRU', 'PSA', 'PSKY', 'PSX', 'PWR', 'PYPL', 'QCOM', 'RCL', 'REG', 'REGN', 'RF', 'RJF', 'RL', 'RMD', 'ROK', 'ROL', 'ROP', 'ROST', 'RSG', 'RTX',
           'RVTY', 'SBAC', 'SBUX', 'SCHW', 'SHW', 'SJM', 'SLB', 'SNA', 'SNPS', 'SO', 'SPG', 'SPGI', 'SRE', 'STE', 'STT', 'STX', 'STZ', 'SW', 'SWK', 'SWKS', 'SYF',
           'SYK', 'SYY', 'T', 'TAP', 'TDG', 'TDY', 'TEL', 'TER', 'TFC', 'TGT', 'TJX', 'TMO', 'TMUS', 'TPR', 'TROW', 'TRV', 'TSCO', 'TSN', 'TT', 'TTWO', 'TXN', 'TXT',
           'TYL', 'UAL', 'UDR', 'UHS', 'ULTA', 'UNH', 'UNP', 'UPS', 'URI', 'USB', 'V', 'VLO', 'VMC', 'VRSK', 'VRSN', 'VRTX', 'VTR', 'VTRS', 'VZ', 'WAB', 'WAT', 'WDC',
           'WEC', 'WELL', 'WFC', 'WM', 'WMB', 'WMT', 'WRB', 'WST', 'WTW', 'WY', 'WYNN', 'XEL', 'XOM', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZTS']

In [20]:
# -----------------------------
# Price adjustments helper
# -----------------------------
def parse_split_factor(factor):
    if factor is None:
        return 1.0
    if isinstance(factor, str) and ":" in factor:
        num, denom = factor.split(":")
        return float(num) / float(denom)
    return float(factor)

# -----------------------------
# Collect and store all data
# -----------------------------

def collect_all_data(tickers, start_date, output_path="all_data.json"):

    all_data = {}

    for tick in tqdm(tickers):
        ticker_obj = Ticker(tick)

        # ---------- PRICE DATA ----------
        prices_df = ticker_obj.price()
        if prices_df is None or prices_df.empty:
            continue
        prices_df["report_date"] = pd.to_datetime(prices_df["report_date"])
        prices_df = prices_df[prices_df["report_date"] >= start_date]
        prices_df.set_index("report_date", inplace=True)
        closes = prices_df["close"].sort_index()
        volumes = prices_df["volume"].sort_index()
        if closes.empty:
            continue

        # Splits
        splits_df = ticker_obj.splits()
        splits_dict = {}
        if splits_df is not None and not splits_df.empty:
            splits_df["report_date"] = pd.to_datetime(splits_df["report_date"])
            splits_df = splits_df[splits_df["report_date"] >= start_date]
            splits_df.set_index("report_date", inplace=True)
            splits_df["split_factor"] = splits_df["split_factor"].apply(parse_split_factor)
            splits_dict = splits_df["split_factor"].to_dict()

        # Dividends
        dividends_df = ticker_obj.dividends()
        dividends_dict = {}
        if dividends_df is not None and not dividends_df.empty:
            dividends_df["report_date"] = pd.to_datetime(dividends_df["report_date"])
            dividends_df = dividends_df[dividends_df["report_date"] >= start_date]
            dividends_df.set_index("report_date", inplace=True)
            dividends_dict = dividends_df["amount"].to_dict()

        # Dividend-adjustment factors
        adj_factor = 1.0
        adj_factors = {}
        for date in reversed(sorted(closes.index)):
            price = float(closes[date])
            div_amt = float(dividends_dict.get(date, 0.0))
            adj_factors[date] = adj_factor
            if div_amt != 0.0 and price > 0:
                adj_factor *= price / (price - div_amt)

        adj_closes = {date: float(closes[date]) / adj_factors[date] for date in closes.index}

        # Base prices DataFrame
        prices_data = pd.DataFrame({
            "symbol": tick,
            "report_date": closes.index,
            "close": closes.values,
            "adj_close": [adj_closes[d] for d in closes.index],
            "adj_factor": [adj_factors[d] for d in closes.index],
            "split_factor": [splits_dict.get(d, 1.0) for d in closes.index],
            "div_amount": [dividends_dict.get(d, 0.0) for d in closes.index],
            "volume": volumes.values
        })

        # ---------- QUARTERLY FUNDAMENTALS ----------
        eps_df = ticker_obj.quarterly_ttm_eps_yoy_growth()
        if eps_df is not None and not eps_df.empty:
            eps_df["report_date"] = pd.to_datetime(eps_df["report_date"])
            eps_df = eps_df[eps_df["report_date"] >= start_date]
            eps_df["symbol"] = tick

            prices_data = prices_data.merge(
                eps_df,
                on=["symbol", "report_date"],
                how="left"
            )

            # Forward-fill EPS values
            prices_data["eps_yoy_growth"] = prices_data.groupby("symbol")["yoy_growth"].ffill()

        prices_data = prices_data.sort_values(["symbol", "report_date"])

        # ---------- FIXED: forward/backward fill ----------
        non_group_cols = [c for c in prices_data.columns if c not in ["symbol", "report_date"]]
        prices_data[non_group_cols] = prices_data.groupby("symbol", group_keys=False)[non_group_cols].apply(lambda x: x.ffill().bfill())

        # ---------- PEG ratio using adj_close ----------
        peg_df = ticker_obj.peg_ratio()
        if peg_df is not None and not peg_df.empty:
            peg_df["report_date"] = pd.to_datetime(peg_df["report_date"])
            prices_data["report_date"] = prices_data["report_date"].dt.normalize()
            peg_df["report_date"] = peg_df["report_date"].dt.normalize()
            peg_df["symbol"] = tick

            peg_merged = peg_df.merge(
                prices_data[["symbol", "report_date", "adj_close"]],
                on=["symbol", "report_date"],
                how="left"
            )

            peg_merged["peg_adj"] = peg_merged["adj_close"] / peg_merged["ttm_eps"] / (peg_merged["eps_yoy_growth"] * 100)
            peg_merged.loc[
                (peg_merged["ttm_eps"].isna()) | (peg_merged["eps_yoy_growth"].isna()) | (peg_merged["eps_yoy_growth"] == 0),
                "peg_adj"
            ] = None

            prices_data = prices_data.merge(
                peg_merged[["symbol", "report_date", "peg_adj"]],
                on=["symbol", "report_date"],
                how="left"
            )
            prices_data["peg_adj"] = prices_data.groupby("symbol")["peg_adj"].ffill()

        prices_data = prices_data.loc[:, ~prices_data.columns.duplicated()]

        # ---------- METADATA ----------
        sector = ticker_obj.info().get("sector")[0]
        prices_data["sector"] = sector

        # ---------- STORE (flat JSON structure) ----------
        ticker_data = {
            "sector": sector,
            "data": []
        }

        for _, row in prices_data.iterrows():
            entry = {
                "date": row["report_date"].strftime("%Y-%m-%d"),
                "close": float(row["close"]),
                "adj_close": float(row["adj_close"]),
                "adj_factor": float(row["adj_factor"]),
                "split_factor": float(row["split_factor"]),
                "div_amount": float(row["div_amount"]),
                "volume": float(row["volume"]),
                "eps_yoy_growth": (
                    None if pd.isna(row.get("eps_yoy_growth")) else float(row["eps_yoy_growth"])
                ),
                "peg_adj": (
                    None if pd.isna(row.get("peg_adj")) else float(row["peg_adj"])
                )
            }
            ticker_data["data"].append(entry)

        all_data[tick] = ticker_data

    # ---------- SAVE JSON ----------
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)

    print(f"All merged data since {start_date.strftime('%Y-%m-%d')} stored in {output_path}")
    
collect_all_data(tickers, start_date, output_path="all_data.json")

100%|█████████████████████████████████████████████████████████████████████████████████| 433/433 [09:22<00:00,  1.30s/it]


All merged data since 2017-12-31 stored in all_data.json


In [ ]:
### IMPORT DATA FROM JSON

In [21]:
with open("all_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

records = []
for ticker, content in data.items():
    sector = content.get("sector", None)
    for rec in content.get("data", []):
        rec["ticker"] = ticker
        rec["sector"] = sector
        # Reorder keys so 'date' appears first
        rec = {"date": rec["date"], **{k: v for k, v in rec.items() if k != "date"}}
        records.append(rec)

df = pd.json_normalize(records)

# Convert 'date' to datetime
df["date"] = pd.to_datetime(df["date"])

# Ensure column order explicitly (in case JSON order changes)
cols = ["date"] + [c for c in df.columns if c != "date"]
df = df[cols]

In [ ]:
###

In [ ]:
# Filter dates from 2020-01-01 onwards
df = df[df["date"] >= "2020-01-01"]

# Count how many unique dates each ticker has
ticker_date_counts = df.groupby("ticker")["report_date"].nunique().reset_index()
ticker_date_counts.rename(columns={"date": "num_dates"}, inplace=True)

# Group tickers by number of dates
grouped = ticker_date_counts.groupby("num_dates")["ticker"].apply(list).reset_index()

# Sort by number of dates
grouped = grouped.sort_values(by="num_dates", ascending=False).reset_index(drop=True)

# Display
for _, row in grouped.iterrows():
    print(f"{row['num_dates']} dates: {row['ticker']}")

In [44]:
# Ensure report_date is datetime
df["date"] = pd.to_datetime(df["date"])

# Define cutoff date
cutoff = pd.Timestamp("2020-01-01")

# Filter records after cutoff
df_filtered = df[df["date"] >= cutoff]

# Count total records per ticker
summary_df = df_filtered.groupby("ticker").size().reset_index(name="total_records")

# Find tickers with maximum records
max_records = summary_df["total_records"].max()
tickers_maxdays = summary_df[summary_df["total_records"] == max_records]["ticker"].tolist()

print(len(tickers_maxdays))
print(tickers_maxdays)

427
['A', 'AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALL', 'ALLE', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BALL', 'BAX', 'BBY', 'BDX', 'BEN', 'BIIB', 'BK', 'BKNG', 'BKR', 'BLK', 'BMY', 'BR', 'BSX', 'BXP', 'C', 'CAG', 'CAH', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COO', 'COP', 'COR', 'COST', 'CPAY', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTRA', 'CTSH', 'CTVA', 'CVS', 'CVX', 'D', 'DAL', 'DAY', 'DD', 'DE', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DLR', 'DLTR', 'DOC', 'DOV', 'DOW', 'DPZ', 'DRI', 'DTE', 'DUK', 'DVA', 'DVN', 'DXCM', 'EA', 'EBAY', 'ECL', 'ED', 'EFX', 'EG', 'EIX', 'EL', 'ELV', 'EMN', 'EMR', 'EOG', 'EQIX'

In [71]:
# Ensure report_date is a datetime
df["report_date"] = pd.to_datetime(df["report_date"])

# Define cutoff date
cutoff = pd.Timestamp("2020-01-01")

# Filter the DataFrame for dates after cutoff
df_filtered = df[df["report_date"] >= cutoff]

# Compute average daily volume per ticker
avg_volume_df = df_filtered.groupby("ticker")["volume"].mean().reset_index()
avg_volume_df.rename(columns={"volume": "avg_volume"}, inplace=True)

# Drop missing values if any
avg_volume_df = avg_volume_df.dropna()

# Sort by average volume and keep top 100
top100_df = avg_volume_df.sort_values("avg_volume", ascending=False).head(100)

# Optional: convert to millions for readability
top100_df["avg_volume_million"] = top100_df["avg_volume"] / 1_000_000
top100_df = top100_df.reset_index(drop=True)

print(top100_df[["ticker", "avg_volume_million"]])

   ticker  avg_volume_million
0    NVDA          417.091831
1    AAPL           85.886052
2       F           71.486191
3    AMZN           65.180831
4     AMD           61.852816
..    ...                 ...
95   NFLX            6.005586
96    DOW            5.970817
97    APH            5.930738
98    ABT            5.802972
99   MCHP            5.802927

[100 rows x 2 columns]


In [22]:
# Top 100 by volume

tickers_maxdays = [
    'NVDA', 'AAPL', 'F', 'AMZN', 'AMD', 'INTC', 'BAC', 'T', 'CCL', 'PFE',
    'GOOGL', 'MSFT', 'GOOG', 'AVGO', 'WFC', 'META', 'WMT', 'XOM', 'CMCSA', 'CSCO',
    'MU', 'VZ', 'C', 'NCLH', 'OXY', 'FCX', 'HBAN', 'CMG', 'KO', 'GM',
    'KMI', 'DAL', 'LRCX', 'UAL', 'CSX', 'HPE', 'KEY', 'SLB', 'JPM', 'PYPL',
    'BMY', 'BA', 'DIS', 'HAL', 'MRK', 'AMCR', 'ORCL', 'GE', 'DVN', 'SCHW',
    'CVX', 'ANET', 'NEE', 'VTRS', 'HPQ', 'RF', 'NKE', 'LUV', 'USB', 'MO',
    'NEM', 'QCOM', 'MS', 'SBUX', 'GILD', 'HST', 'CVS', 'APA', 'JNJ', 'KHC',
    'WMB', 'BSX', 'BKR', 'TFC', 'COP', 'CTRA', 'MGM', 'V', 'AES', 'EXC',
    'AMAT', 'PG', 'ORLY', 'MDLZ', 'KR', 'CRM', 'EBAY', 'FAST', 'WDC', 'ABBV',
    'RTX', 'LVS', 'TJX', 'FTNT', 'MDT', 'NFLX', 'DOW', 'APH', 'ABT', 'MCHP'
]

In [23]:
# -------------------------------
# Containers for ticker data
# -------------------------------
sectors = []
industries = []
market_caps = []

# -------------------------------
# Fetch info and summary for all tickers
# -------------------------------
for symbol in tickers_maxdays:
    try:
        ticker = Ticker(symbol)
        info = ticker.info()[["sector", "industry"]].to_dict(orient="records")[0]
        summary = ticker.summary().to_dict(orient="records")[0]

        sectors.append(info.get("sector"))
        market_caps.append(summary.get("market_cap"))
    except Exception as e:
        print(f"Skipping {symbol}: {e}")
        sectors.append(None)
        industries.append(None)
        market_caps.append(None)

ticker_info_df = pd.DataFrame({
    "ticker": tickers_maxdays,
    "sector": sectors,
})

market_cap_df = pd.DataFrame({
    "ticker": tickers_maxdays,
    "market_cap": market_caps
}).dropna()

In [37]:
# -------------------------------
# 1. Count of stocks by sector
# -------------------------------
# Count of stocks by sector
sector_count = ticker_info_df["sector"].value_counts().reset_index()
sector_count.columns = ["sector", "count"]

fig_sector = px.bar(
    sector_count,
    x="sector",
    y="count",
    text="count",
    color_discrete_sequence=["#4C72B0"]
)

fig_sector.update_layout(
    showlegend=False,
    font=dict(family="Calibri", size=20),
    plot_bgcolor="white",
    paper_bgcolor="white",
    margin=dict(t=10),
    xaxis=dict(showgrid=False, showline=True, linecolor="black"),
    yaxis=dict(showgrid=True, gridcolor="#D3D3D3", zeroline=False, showline=True, linecolor="black"),
    xaxis_title="Sector",
    yaxis_title="Number of stocks"
)

fig_sector.update_traces(textposition="outside")
fig_sector.write_html("sector_distribution.html")

In [38]:
# -------------------------------
# 2. Market capitalization distribution (log-scaled)
# -------------------------------

# Filter positive market caps and log-transform
market_cap_df = market_cap_df[market_cap_df["market_cap"] > 0].copy()
market_cap_df["log_market_cap"] = np.log10(market_cap_df["market_cap"])

fig_market_cap = px.histogram(
    market_cap_df,
    x="log_market_cap",
    nbins=80,
    color_discrete_sequence=["#C44E52"]
)

fig_market_cap.update_layout(
    showlegend=False,
    font=dict(family="Calibri", size=20),
    plot_bgcolor="white",
    paper_bgcolor="white",
    margin=dict(t=10),
    xaxis=dict(showgrid=False, showline=True, linecolor="black"),
    yaxis=dict(showgrid=True, gridcolor="#D3D3D3", zeroline=False, showline=True, linecolor="black"),
    xaxis_title="Log of market capitalization (USD)",
    yaxis_title="Number of stocks"
)

fig_market_cap.write_html("market_cap_distribution.html")

In [41]:
# -------------------------------
# 3. Daily volume
# -------------------------------

# Ensure report_date is datetime
df["date"] = pd.to_datetime(df["date"])

# Filter by date and selected tickers
cutoff = pd.Timestamp("2020-01-01")
df_filtered = df[(df["date"] >= cutoff) & (df["ticker"].isin(tickers_maxdays))]

# Compute average daily volume in millions
avg_volume_df = (
    df_filtered.groupby("ticker")["volume"]
    .mean()
    .div(1_000_000)
    .reset_index()
    .rename(columns={"volume": "avg_volume_million"})
)

# Select top 100 tickers by average volume
top100_df = avg_volume_df.sort_values("avg_volume_million", ascending=False).head(100)

fig_volume = px.bar(
    top100_df,
    x="ticker",
    y="avg_volume_million",
    labels={"avg_volume_million": "Avg. daily vol. (USDm)", "ticker": "Ticker"},
    color="avg_volume_million",
    color_continuous_scale=px.colors.sequential.Plasma
)

fig_volume.update_layout(
    font=dict(family="Calibri", size=20),
    plot_bgcolor="white",
    paper_bgcolor="white",
    margin=dict(t=10),
    xaxis=dict(showgrid=True, gridcolor="#E5E5E5", tickangle=45),
    yaxis=dict(showgrid=True, gridcolor="#E5E5E5", zeroline=False, showline=True, linecolor="black", type="log")
)

fig_volume.write_html("avg_daily_volume_2020_log.html")

In [42]:
# -------------------------------
# 4. Sector by volume
# -------------------------------

# Merge top 100 tickers with sector info
top100_sector_df = top100_df.merge(df[["ticker", "sector"]], on="ticker", how="left")

# Compute total average daily volume per sector
sector_volume = top100_sector_df.groupby("sector")["avg_volume_million"].agg(['sum', 'mean', 'count']).reset_index()
sector_volume = sector_volume.sort_values("sum", ascending=False)

fig_sector_volume = px.bar(
    sector_volume,
    x="sector",
    y="sum",
    color_discrete_sequence=["#4C72B0"]
)

fig_sector_volume.update_layout(
    showlegend=False,
    font=dict(family="Calibri", size=20),
    plot_bgcolor="white",
    paper_bgcolor="white",
    margin=dict(t=10),
    xaxis=dict(showgrid=False, showline=True, linecolor="black", tickangle=45),
    yaxis=dict(showgrid=True, gridcolor="#D3D3D3", zeroline=False, showline=True, linecolor="black"),
    xaxis_title="Sector",
    yaxis_title="Total avg. daily vol. (USDm)"
)

fig_sector_volume.update_traces(textposition="outside")
fig_sector_volume.write_html("top_sectors_by_avg_volume_academic.html")